In [167]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import re

In [165]:
def convert_date(x):
    # Ensure x is a string
    if not isinstance(x, str):
        x = str(x)
    
    try:
        # Handle patterns like "YYYY/YY" (e.g. "2023/24")
        match_slash = re.search(r'(\d{4})/(\d{2})', x)
        if match_slash:
            full_year = match_slash.group(1)
            short_year = match_slash.group(2)
            year_candidate = int(full_year[:2] + short_year)
            if not (1900 <= year_candidate <= 2262):
                return pd.NaT
            return pd.to_datetime(f"{year_candidate}-01-01")
        
        # Handle quarter patterns like "YYYYQ[1-4]" (e.g. "2024Q4")
        match_quarter = re.search(r'(\d{4})Q([1-4])', x)
        if match_quarter:
            year_candidate = int(match_quarter.group(1))
            if not (1900 <= year_candidate <= 2262):
                return pd.NaT
            return pd.to_datetime(f"{year_candidate}-01-01")
        
        # Handle month patterns like "YYYYM\d{2}" (e.g. "2004M01" or "2010M12")
        match_month = re.search(r'(\d{4})M(\d{2})', x)
        if match_month:
            year_candidate = int(match_month.group(1))
            if not (1900 <= year_candidate <= 2262):
                return pd.NaT
            return pd.to_datetime(f"{year_candidate}-01-01")
        
        # Fallback: Extract all 4-digit numbers
        years = re.findall(r"\d{4}", x)
        if years:
            # Filter out years that are not in a reasonable range
            valid_years = [int(y) for y in years if 1900 <= int(y) <= 2262]
            if valid_years:
                max_year = max(valid_years)
                return pd.to_datetime(f"{max_year}-01-01")
    except (ValueError, OverflowError, pd.errors.OutOfBoundsDatetime):
        return pd.NaT
    
    return pd.NaT

In [166]:
# Load the DataFrame
df = pd.read_pickle("cso_table.pkl")

# Convert 'latest' and 'earliest' using the convert_date function
df[['latest', 'earliest']] = df[['latest', 'earliest']].applymap(convert_date)

# Drop rows with missing dates in either column
df.dropna(subset=['latest', 'earliest'], inplace=True)

# Define the cutoff date and filter rows
cutoff_date = pd.to_datetime("2022-01-01")
df = df.loc[df['latest'] >= cutoff_date].copy()

# Convert lists in 'variables' to tuples of lowercase strings
df['variables_tuple'] = df['variables'].apply(lambda x: tuple(item.lower() for item in x))

# Create a new column with the lowercase version of 'table_name'
df['table_name_lower'] = df['table_name'].str.lower()

# Sort: 'latest' descending, 'earliest' ascending
df.sort_values(by=['latest', 'earliest'], ascending=[False, True], inplace=True)

# Drop duplicates based on the lowercased 'variables' and 'table_name'
result_df = df.drop_duplicates(subset=['variables_tuple', 'table_name_lower'], keep='first')

C:\Users\James\AppData\Local\Temp\ipykernel_17524\4084037481.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['latest', 'earliest']] = df[['latest', 'earliest']].applymap(convert_date)


In [169]:
# CONFIGURATION
INDEX_PATH  = "vector_index.faiss"
META_PATH   = "metadata.pkl"
MODEL_NAME  = "sentence-transformers/all-MiniLM-L6-v2"
EMBED_DIM   = 384  # model output dimension

# Assume your DataFrame is already loaded as "table"
# Use the 'table_name' column (or any desired column) as the document text.
docs = result_df['table_name'].astype(str).tolist()


metadata = result_df.apply(lambda row: {
    "table_id": str(row.name),
    "table_name": row["table_name"],
    "fields": row["variables"],
    "earliest": str(row["earliest"].year),
    "latest": str(row["latest"].year),  # added latest
    "frequency": row["frequency"],
}, axis=1).tolist()

# 1 Embed the documents
model = SentenceTransformer(MODEL_NAME)
embeddings = model.encode(docs, show_progress_bar=True)

# 2 Build the FAISS index
index = faiss.IndexFlatL2(EMBED_DIM)
index.add(embeddings)

# 3 Save the index and metadata
faiss.write_index(index, INDEX_PATH)
with open(META_PATH, "wb") as f:
    pickle.dump(metadata, f)

print(f"Indexed {len(docs)} docs → {INDEX_PATH}")


Batches:   0%|          | 0/118 [00:00<?, ?it/s]

Indexed 3753 docs → vector_index.faiss
